In [4]:
# !python3 -m pip install matplotlib

In [8]:
# !python3 -m pip install pandas

In [7]:
# !python3 -m pip install mysql-connector

In [9]:
# !python3 -m pip install pymysql

In [12]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

ts = TimeSeries(key='GWXLIMM8PJ1W2F2C', output_format='json')
data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
print(data)
print(meta_data)
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# plt.show()

{'2021-11-05 19:52:00': {'1. open': '335.8300', '2. high': '335.8300', '3. low': '335.8300', '4. close': '335.8300', '5. volume': '584'}, '2021-11-05 19:51:00': {'1. open': '335.8800', '2. high': '335.8800', '3. low': '335.8800', '4. close': '335.8800', '5. volume': '200'}, '2021-11-05 19:49:00': {'1. open': '335.9100', '2. high': '335.9100', '3. low': '335.9100', '4. close': '335.9100', '5. volume': '393'}, '2021-11-05 19:36:00': {'1. open': '335.9000', '2. high': '335.9000', '3. low': '335.9000', '4. close': '335.9000', '5. volume': '110'}, '2021-11-05 19:32:00': {'1. open': '336.0000', '2. high': '336.0000', '3. low': '336.0000', '4. close': '336.0000', '5. volume': '228'}, '2021-11-05 19:29:00': {'1. open': '336.0000', '2. high': '336.0000', '3. low': '336.0000', '4. close': '336.0000', '5. volume': '1000'}, '2021-11-05 19:25:00': {'1. open': '335.9999', '2. high': '335.9999', '3. low': '335.9999', '4. close': '335.9999', '5. volume': '107'}, '2021-11-05 18:54:00': {'1. open': '336

## GET API

In [8]:
# key = 'GWXLIMM8PJ1W2F2C'

In [1]:
## FUNCTION
import pymysql

api_url = 'https://www.alphavantage.co/query'
    
def get_alphavantage(params_dict: dict()) -> dict():
    params = '&'.join([f'{key}={value}' for key, value in params_dict.items()])
    print(params)
    
    import requests
    
    url = f'{api_url}?{params}'
    r = requests.get(url)
    data = r.json()
    return data

def init_db() -> pymysql.connections.Connection:
    # Open database connection
    # Connect to the database
    db = pymysql.connect(
        host='alphavantage.ckhcoakuhojq.ap-southeast-1.rds.amazonaws.com',
         user='admin',
         password='adminadmin',
         database='alphavantage_uat',
         cursorclass=pymysql.cursors.DictCursor
    )
    return db

def insert_data(df, table_name, db):
    # prepare a cursor object using cursor() method
    cursor = db.cursor()
    
    field_len = len(df.columns.values)
    cols = ','.join(list(df.columns.values))

    for i, row in df.iterrows():
        print(i)
        data_row = ','.join([row[list(df.columns.values)[field_i]] for field_i in range(field_len)])

        sql = f"""
            INSERT IGNORE INTO {table_name} ({cols})
            VALUES ({data_row});
        """
        print(sql.format(table_name,cols,data_row))

        cursor.execute(sql)
        
db = init_db()

In [4]:
## daily ticker/currency (/api/dailyeod)
params_dict = {
    'function': 'FX_DAILY',
    'from_symbol': 'EUR',
    'to_symbol': 'USD',
    'apikey': 'demo'
}
data = get_alphavantage(params_dict)
# data

# df = pd.read_json(fname).reset_index()
df = pd.DataFrame(data).reset_index()

df = pd.concat([
    df[df.columns.tolist()[-1]].dropna().apply(pd.Series),
    df.drop(df.columns.tolist()[-1], axis=1)
], axis=1)

df = df[
    df[df.columns.tolist()[-1]].isnull() # take out last column with null value (Meta Data) - logic will change if json structure changes
].drop(
    df.columns.tolist()[-1], axis=1 # drop empty column (Meta Data)
)
df.rename(columns={'index': 'date'}, inplace=True)
df.columns = [x.replace('.','_').replace(' ','').lower() for x in df.columns]
df['date'] = df['date'].apply(lambda x: f"'{x}'")

df = df.reset_index(drop=True)
# df

table_name = '_'.join(list(params_dict.values())[:-1]).lower()
print(table_name)

insert_data(df, table_name, db)

function=FX_DAILY&from_symbol=EUR&to_symbol=USD&apikey=demo
fx_daily_eur_usd
0

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.15865,1.16065,1.15750,1.15840,'2021-11-09');
        
1

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.15573,1.15950,1.15490,1.15863,'2021-11-08');
        
2

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.15539,1.15732,1.15120,1.15660,'2021-11-05');
        
3

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.16122,1.16164,1.15260,1.15526,'2021-11-04');
        
4

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.15781,1.16164,1.15600,1.16130,'2021-11-03');
        
5

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.16032

96

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.19352,1.19445,1.19000,1.19246,'2021-06-28');
        
97

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.19300,1.19753,1.19240,1.19330,'2021-06-25');
        
98

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.19272,1.19563,1.19160,1.19303,'2021-06-24');
        
99

            INSERT IGNORE INTO fx_daily_eur_usd (1_open,2_high,3_low,4_close,date)
            VALUES (1.19399,1.19701,1.19100,1.19273,'2021-06-23');
        


In [ ]:
## monthly CPI (/api/monthlycpi)
params_dict = {
    'function': 'CPI',
    'interval': 'monthly',
    'apikey': 'demo'
}
data = get_alphavantage(params_dict)
# data

# df = pd.read_json(fname)
df = pd.DataFrame(data)

df = pd.concat([
    df[df.columns.tolist()[-1]].apply(pd.Series),
    df.drop(df.columns.tolist()[-1], axis=1)
], axis=1)
df = df[['date','value']]
df['date'] = df['date'].apply(lambda x: f"'{x}'")

df = df.reset_index(drop=True)
# df

table_name = '_'.join(list(params_dict.values())[:-1]).lower()
print(table_name)

insert_data(df, table_name, db)

## CLEAN DATA

## QUERY

In [5]:
import pandas as pd

query = f"SELECT * FROM fx_daily_eur_usd;"
df = pd.read_sql(query, db)
df

,date,1_open,2_high,3_low,4_close
0,2021-06-23,1.19399,1.19701,1.1910,1.19273
1,2021-06-24,1.19272,1.19563,1.1916,1.19303
2,2021-06-25,1.19300,1.19753,1.1924,1.19330
3,2021-06-28,1.19352,1.19445,1.1900,1.19246
4,2021-06-29,1.19258,1.19300,1.1876,1.18982
...,...,...,...,...,...
95,2021-11-03,1.15781,1.16164,1.1560,1.16130
96,2021-11-04,1.16122,1.16164,1.1526,1.15526
97,2021-11-05,1.15539,1.15732,1.1512,1.15660
98,2021-11-08,1.15573,1.15950,1.1549,1.15863


In [6]:
import pandas as pd

query = f"SELECT * FROM cpi_monthly;"
df = pd.read_sql(query, db)
df

,date,value


## CREATE TABLE - Adhoc

In [9]:
# cursor = db.cursor()
    
# # Drop table if it already exist using execute() method.
# cursor.execute(f"DROP TABLE IF EXISTS fx_daily_eur_usd;")

# # Create table as per requirement
# sql = f"""
#     CREATE TABLE IF NOT EXISTS fx_daily_eur_usd (
#         date DATE PRIMARY KEY,
#         1_open FLOAT,
#         2_high FLOAT,
#         3_low FLOAT,
#         4_close FLOAT
#     );
# """

# cursor.execute(sql)

In [6]:
# cursor = db.cursor()

# # Drop table if it already exist using execute() method.
# cursor.execute(f"DROP TABLE IF EXISTS cpi_monthly;")

# # Create table as per requirement
# sql = f"""
#     CREATE TABLE IF NOT EXISTS cpi_monthly (
#         date DATE PRIMARY KEY,
#         value FLOAT
#     );
# """

# cursor.execute(sql)

0

In [44]:
# cursor.execute("SHOW TABLES;")

# for tables in cursor:
#     print(tables)

{'Tables_in_alphavantage_uat': 'cpi_monthly'}
{'Tables_in_alphavantage_uat': 'fx_daily_eur_usd'}
{'Tables_in_alphavantage_uat': 'table_name'}


In [33]:
# cursor.execute(f"DESCRIBE {table_name};")

# for field in cursor:
#     print(field)

{'Field': 'date', 'Type': 'date', 'Null': 'YES', 'Key': '', 'Default': None, 'Extra': ''}
{'Field': 'value', 'Type': 'float', 'Null': 'YES', 'Key': '', 'Default': None, 'Extra': ''}


## MONTHLY

In [11]:
# import requests

# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey={key}'
# r = requests.get(url)
# data = r.json()

# print(data)

{'name': 'Consumer Price Index for all Urban Consumers', 'interval': 'monthly', 'unit': 'index 1982-1984=100', 'data': [{'date': '2021-09-01', 'value': '274.310'}, {'date': '2021-08-01', 'value': '273.567'}, {'date': '2021-07-01', 'value': '273.003'}, {'date': '2021-06-01', 'value': '271.696'}, {'date': '2021-05-01', 'value': '269.195'}, {'date': '2021-04-01', 'value': '267.054'}, {'date': '2021-03-01', 'value': '264.877'}, {'date': '2021-02-01', 'value': '263.014'}, {'date': '2021-01-01', 'value': '261.582'}, {'date': '2020-12-01', 'value': '260.474'}, {'date': '2020-11-01', 'value': '260.229'}, {'date': '2020-10-01', 'value': '260.388'}, {'date': '2020-09-01', 'value': '260.280'}, {'date': '2020-08-01', 'value': '259.918'}, {'date': '2020-07-01', 'value': '259.101'}, {'date': '2020-06-01', 'value': '257.797'}, {'date': '2020-05-01', 'value': '256.394'}, {'date': '2020-04-01', 'value': '256.389'}, {'date': '2020-03-01', 'value': '258.115'}, {'date': '2020-02-01', 'value': '258.678'}, 

## CONNECT TO MySQL RDS

In [295]:
import pymysql

In [296]:
# Connect to the database
connection = pymysql.connect(host='alphavantage.ckhcoakuhojq.ap-southeast-1.rds.amazonaws.com',
                             user='admin',
                             password='adminadmin')
#                              database='alphavantage',
#                              cursorclass=pymysql.cursors.DictCursor)
connection

In [297]:
cursor = connection.cursor()
cursor

In [304]:
# execute SQL query using execute() method.
cursor.execute("SELECT VERSION();")

# Fetch a single row using fetchone() method.
ver = cursor.fetchone()
print ("Database version : %s " % ver)

Database version : 8.0.23 


## CREATE DATABASE

In [47]:
connection.cursor().execute('CREATE DATABASE alphavantage_uat;')

1

## SHOW DATABASES

In [299]:
cursor.execute("SHOW DATABASES;")

for databases in cursor:
    print(databases[0])

alphavantage
alphavantage_uat
information_schema
mysql
performance_schema
sys
